In [ ]:
# Import TensorFlow and hub
import tensorflow as tf
import tensorflow_hub as hub


# some important packages
import os
import re
import numpy as np
import pandas as pd


# Carregar o modelo Universal Sentence Encoder
model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(model_url)
print('Model Loaded')

# Função para fazer embedding dos textos
def embed(texts):
    return model(texts)

# Testando a função de embedding
embed(["This movie was the worst movie i've seen!"])

# Carregar o dataset de filmes (ajuste o caminho conforme necessário)
df = pd.read_csv(r"/Top_10000_Movies.csv", engine="python")
df.head()

# Selecionando as colunas relevantes
df = df[["original_title", "overview"]]
df.head()

# Remover valores ausentes e resetar o índice
df = df.dropna()
df = df.reset_index()
df = df[:10000]  # Limitar a 10.000 filmes, caso necessário

# Listar as descrições dos filmes
titles = list(df['overview'])

# Exibir as primeiras descrições
titles[:8]

# Gerar os embeddings para todas as descrições
embeddings = embed(titles)
print('The embedding shape is:', embeddings.shape)

# Redução de dimensionalidade com PCA para visualização
pca = PCA(n_components=2)
emb_2d = pca.fit_transform(embeddings)

# Treinando o modelo de vizinhos mais próximos
nn = NearestNeighbors(n_neighbors=10)
nn.fit(embeddings)

# Função para recomendar filmes semelhantes com base em uma descrição
def recommend(text):
    emb = embed([text])
    neighbors = nn.kneighbors(emb, return_distance=False)[0]
    return df['original_title'].iloc[neighbors].tolist()

# Testando a recomendação com um filme
print('Recommended Movies:')
# Solicitar ao usuário o nome do filme
user_input = input("Digite o nome do filme: ")

# Obter recomendações com base na entrada do usuário
recommended_movies = recommend(user_input)

# Exibir as recomendações
print("Recommended Movies:")
for movie in recommended_movies:
    print(movie)

